In [4]:
import requests
import json

requets = requests.get

In [24]:
import requests
import json
import math

# --- CONFIGURATION ---
OVERPASS_URL = "http://overpass-api.de/api/interpreter"
# Large timeout because Finland is a large area
OVERPASS_QUERY = """
[out:json][timeout:900];
area["ISO3166-1"="FI"]->.searchArea;
(
  relation["route"="hiking"](area.searchArea);
  relation["route"="foot"](area.searchArea);
);
out geom;
"""

def haversine_distance(coord1, coord2):
    """Calculate distance between two lat/lon points in meters."""
    R = 6371000
    phi1 = math.radians(coord1['lat'])
    phi2 = math.radians(coord2['lat'])
    dphi = math.radians(coord2['lat'] - coord1['lat'])
    dlambda = math.radians(coord2['lon'] - coord1['lon'])

    a = math.sin(dphi / 2)**2 + \
        math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

def get_photo_links(tags):
    """Extracts photo urls from tags."""
    links = []
    keys_to_check = ['image', 'url', 'website', 'mapillary']
    for key in keys_to_check:
        if key in tags:
            links.append(tags[key])
    
    # Special handling for Wikimedia Commons tags
    if 'wikimedia_commons' in tags:
        val = tags['wikimedia_commons']
        if val.startswith("Category:") or val.startswith("File:"):
            links.append(f"https://commons.wikimedia.org/wiki/{val}")
        else:
            links.append(val)
            
    return links

def get_description(tags):
    """
    Tries to find a description in this order:
    1. Generic description
    2. English description
    3. Finnish description
    """
    if 'description' in tags:
        return tags['description']
    if 'description:en' in tags:
        return tags['description:en']
    if 'description:fi' in tags:
        return tags['description:fi']
    # Sometimes descriptions are hidden in 'note' (though usually for mappers)
    # or 'comment', but we stick to official description tags for quality.
    return None

def process_hikes():
    print("Sending query to Overpass API...")
    try:
        response = requests.get(OVERPASS_URL, params={'data': OVERPASS_QUERY})
        response.raise_for_status()
        data = response.json()
    except Exception as e:
        print(f"Error fetching data: {e}")
        return

    hikes_list = []
    elements = data.get('elements', [])
    print(f"Processing {len(elements)} elements...")

    for element in elements:
        if element['type'] == 'relation':
            tags = element.get('tags', {})
            members = element.get('members', [])
            
            # 1. BASIC INFO
            osm_id = element['id']
            name = tags.get('name', "Unnamed Hike")
            
            # 2. DESCRIPTION (New)
            description = get_description(tags)
            
            # 3. CALCULATE GEOMETRY
            total_length_meters = 0
            coordinates_list = [] 

            for member in members:
                if member['type'] == 'way' and 'geometry' in member:
                    points = member['geometry']
                    segment_coords = []
                    for i in range(len(points) - 1):
                        dist = haversine_distance(points[i], points[i+1])
                        total_length_meters += dist
                        segment_coords.append([points[i]['lon'], points[i]['lat']])
                    
                    if points:
                        last = points[-1]
                        segment_coords.append([last['lon'], last['lat']])
                    
                    coordinates_list.append(segment_coords)

            length_km = round(total_length_meters / 1000, 2)

            # 4. GATHER ASSETS
            photos = get_photo_links(tags)
            
            links_dict = {
                "view_on_osm": f"https://www.openstreetmap.org/relation/{osm_id}",
                "download_xml_api": f"https://www.openstreetmap.org/api/0.6/relation/{osm_id}/full",
                "waymarked_trails": f"https://hiking.waymarkedtrails.org/#route?id={osm_id}"
            }

            geojson_obj = {
                "type": "Feature",
                "properties": {
                    "name": name,
                    "osm_id": osm_id,
                    "description": description
                },
                "geometry": {
                    "type": "MultiLineString",
                    "coordinates": coordinates_list
                }
            }

            # 5. ELEVATION CHECK
            # Sometimes manually tagged as 'ascent' or 'descent'
            elevation_info = None
            if 'ascent' in tags:
                elevation_info = f"Ascent: {tags['ascent']}"
            else:
                elevation_info = "No elevation data (requires external DEM)"

            # BUILD ENTRY
            hike_entry = {
                "id": osm_id,
                "name": name,
                "description": description, # Added here
                "length_km": length_km,
                "photos": photos,
                "links": links_dict,
                "elevation": elevation_info,
                "geojson": geojson_obj
            }

            hikes_list.append(hike_entry)

    # Save to file
    output_filename = "finland_hikes.json"
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(hikes_list, f, ensure_ascii=False, indent=4)

    print(f"Done. {len(hikes_list)} hikes saved to {output_filename}")

if __name__ == "__main__":
    process_hikes()

Sending query to Overpass API...
Processing 833 elements...
Done. 833 hikes saved to finland_hikes.json
